<a href="https://colab.research.google.com/github/AyeshaAnzerBCIT/Multisource/blob/main/Fusion_preprocess_(EEG).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import torch
import scipy.io
import h5py
import pandas as pd
import matplotlib.pyplot as plt
import torchaudio.transforms as T
from scipy.stats import entropy
from tqdm import tqdm
import scipy.signal
import numpy as np
import gc
from scipy.signal import welch  #  Import Welch's method for PSD computation
from scipy.stats import entropy

In [ ]:
# --- SET DEVICE ---
# Force the use of GPU 1
device = torch.device("cuda:0" if torch.cuda.device_count() > 1 else "cuda:0")

print(f" Using device: {device}")

 Using device: cuda:0


In [ ]:
# --- STEP 1: FIND ALL EEG FILES ---
def find_mat_files(root_dir, modality="EEG"):
    """Finds all .mat files under the given modality folder."""
    mat_files = []
    for subdir, _, files in os.walk(root_dir):
        if modality in subdir and "mat_format" in subdir:
            for file in files:
                if file.endswith(".mat"):
                    mat_files.append(os.path.join(subdir, file))
    return mat_files

root_dataset_path = "/data/ayesha/PhD/dataset/EEG"
mat_files = find_mat_files(root_dataset_path)
print(f"Total .mat files found: {len(mat_files)}")

Total .mat files found: 1392


In [ ]:
#Step 2
def load_mat_file(file_path):
    """Loads EEG data from MATLAB v7.3 .mat files using h5py."""
    try:
        with h5py.File(file_path, 'r') as f:
            data = {}

            if 'EEG' in f:
                eeg_group = f['EEG']

                if 'data' in eeg_group:
                    eeg_data = eeg_group['data'][()]
                    eeg_data = torch.tensor(eeg_data, dtype=torch.float32).to(device)

                    if eeg_data.shape[1] > eeg_data.shape[0]:
                        eeg_data = eeg_data.T

                    data['EEG'] = eeg_data
                    print(f" Loaded EEG data from {file_path} with shape {eeg_data.shape}")  # Debug print

                if 'srate' in eeg_group:
                    srate = eeg_group['srate'][()]
                    if srate.shape == (1, 1):  # Extract scalar value
                        srate = srate[0, 0]
                    data['srate'] = torch.tensor(srate, dtype=torch.float32).to(device)
                else:
                    data['srate'] = torch.tensor(256.0, dtype=torch.float32).to(device)

            return data if 'EEG' in data else None
    except Exception as e:
        print(f"⚠ Error loading {file_path}: {e}")
        return None

In [ ]:
# --- STEP 3: CHECK IF EEG DATA IS PREPROCESSED ---
def is_preprocessed(file_path):
    """Checks if the EEG data is already preprocessed."""
    return "preprocessed" in file_path.lower()

In [ ]:
# --- STEP 4: TORCH-BASED EEG SIGNAL PREPROCESSING ---

def bandpass_filter_torch(data, lowcut=0.5, highcut=50.0, fs=256, order=5):
    """Applies a bandpass filter using SciPy on GPU 1."""
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = scipy.signal.butter(order, [low, high], btype='band')

    filtered_np = scipy.signal.filtfilt(b, a, data.cpu().numpy(), axis=1)
    filtered_np = np.ascontiguousarray(filtered_np)

    return torch.tensor(filtered_np, dtype=torch.float32, device=device)  # Move to GPU 1

def normalize_signal_torch(data):
    """Normalizes EEG signals on GPU 1."""
    return ((data - data.min()) / (data.max() - data.min()) * 2 - 1).to(device)


In [ ]:
# --- STEP 5: PREPROCESS EEG ---


def preprocess_eeg(mat_files, save_path="/data/ayesha/PhD/dataset/EEG/preprocessed"):
    """Preprocesses EEG signals and saves them with memory optimizations."""
    os.makedirs(save_path, exist_ok=True)
    preprocessed_data = {}

    print(f"Processing {len(mat_files)} EEG files...")

    for file_path in tqdm(mat_files, desc=" Processing EEG", unit="file"):
        data = load_mat_file(file_path)

        if data and "EEG" in data:
            eeg_signal = data["EEG"].squeeze()
            srate = data.get("srate", torch.tensor(256)).item()

            # Apply preprocessing
            filtered_signal = bandpass_filter_torch(eeg_signal, fs=srate)
            normalized_signal = normalize_signal_torch(filtered_signal)

            subject_id = os.path.basename(file_path).split(".")[0]
            torch.save(normalized_signal, os.path.join(save_path, f"{subject_id}_preprocessed.pt"))

            #  Free memory after each file
            del filtered_signal, normalized_signal, data
            torch.cuda.empty_cache()  # Release GPU memory
            gc.collect()  # Free Python memory

    print(" Preprocessing Complete!")
    return preprocessed_data


In [ ]:
def compute_psd_torch(data, fs=256):
    """Compute Power Spectral Density (PSD) using Welch’s method."""
    psd_list = []
    freq_list = []

    for channel in data:  # Iterate over EEG channels
        if isinstance(channel, torch.Tensor):
            channel = channel.cpu().numpy()  #  Ensure it's a NumPy array

        freqs, psd = welch(channel, fs, nperseg=fs)
        psd_list.append(psd)
        freq_list.append(freqs)

    return np.array(psd_list), np.array(freq_list)

def hjorth_parameters(eeg_signal):
    """Compute Hjorth Mobility and Complexity."""
    variance = np.var(eeg_signal, axis=1)  # Variance per channel
    first_derivative = np.diff(eeg_signal, axis=1)
    variance_derivative = np.var(first_derivative, axis=1)

    mobility = np.sqrt(variance_derivative / variance)
    complexity = np.sqrt(np.var(np.diff(first_derivative, axis=1), axis=1) / variance_derivative) / mobility

    return mobility, complexity

def spectral_entropy(eeg_signal, fs=256):
    """Compute Spectral Entropy from EEG signal."""
    psd_values, _ = compute_psd_torch(eeg_signal, fs)
    psd_normalized = psd_values / np.sum(psd_values, axis=1, keepdims=True)  # Normalize PSD
    return np.apply_along_axis(entropy, 1, psd_normalized, base=2)


In [ ]:
# --- STEP 7: SAVE FEATURES ---
def save_features(preprocessed_data, save_path="/data/ayesha/PhD/dataset/EEG/features"):
    """Extracts and saves EEG features from preprocessed .pt files, avoiding duplicate processing."""
    os.makedirs(save_path, exist_ok=True)

    # ? Get list of already processed files
    processed_files = {f.replace("_features.csv", "") for f in os.listdir(save_path) if f.endswith(".csv")}

    # ? Filter only unprocessed EEG files
    unprocessed_data = {subj: path for subj, path in preprocessed_data.items() if subj not in processed_files}

    if not unprocessed_data:
        print("? All EEG files have already been processed! No new processing required.")
        return

    print(f"?? Extracting features for {len(unprocessed_data)} NEW subjects...")

    for subject_id, file_path in tqdm(unprocessed_data.items(), desc=" Extracting Features", unit="subject"):
        try:
            # ? Load EEG data
            eeg_data = torch.load(file_path, map_location="cpu")

            if not isinstance(eeg_data, torch.Tensor):
                print(f"?? Warning: Data in {file_path} is not a tensor.")
                continue

            print(f"?? Processing {subject_id} - EEG Shape: {eeg_data.shape}")  # Debugging

            # Compute EEG Features
            psd_values, _ = compute_psd_torch(eeg_data)
            mobility, complexity = hjorth_parameters(eeg_data.numpy())  # Convert to NumPy
            entropy_values = spectral_entropy(eeg_data.numpy())

            # Organize Features
            feature_dict = {
                "Hjorth_Mobility": mobility.tolist(),
                "Hjorth_Complexity": complexity.tolist(),
                "Spectral_Entropy": entropy_values.tolist()
            }

            df_features = pd.DataFrame(feature_dict)
            df_features.to_csv(os.path.join(save_path, f"{subject_id}_features.csv"), index=False)

        except Exception as e:
            print(f"? Error processing {subject_id}: {e}")

    print("? Feature Extraction Complete!")


In [ ]:
# ------------------------------ #
# Run the Updated Pipeline
# ------------------------------ #

print("?? Starting PyTorch-Based EEG Preprocessing & Feature Extraction...\n")

unzipped_path = "/data/ayesha/PhD/dataset/EEG/preprocessed_unzipped"
preprocessed_data = {
    f.replace("_preprocessed.pt", ""): os.path.join(unzipped_path, f)
    for f in os.listdir(unzipped_path) if f.endswith(".pt")
}

if not preprocessed_data:
    print("?? No EEG .pt files found! Check dataset path and structure.")
else:
    save_features(preprocessed_data)  # ? Process only new files

?? Starting PyTorch-Based EEG Preprocessing & Feature Extraction...

?? Extracting features for 201 NEW subjects...


/tmp/ipykernel_1404530/2335674740.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  eeg_data = torch.load(file_path, map_location="cpu")


?? Processing gip_A00053990019 - EEG Shape: torch.Size([149903, 111])


/home/ayesha/miniconda3/envs/jupyter_env/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:600: UserWarning: nperseg = 256 is greater than input length  = 111, using nperseg = 111
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
 Extracting Features:   0%|              | 1/201 [00:48<2:40:19, 48.10s/subject]

?? Processing gp_A00055837001 - EEG Shape: torch.Size([200521, 111])


...

?? Processing gip_A00054917003 - EEG Shape: torch.Size([334357, 111])


 Extracting Features: 100%|███████████▉| 200/201 [2:34:43<01:19, 79.82s/subject]

?? Processing gip_A00062329004 - EEG Shape: torch.Size([215630, 111])


 Extracting Features: 100%|████████████| 201/201 [2:35:55<00:00, 46.55s/subject]

? Feature Extraction Complete!


In [ ]:
def unzip_pt_files(preprocessed_path):
    """Loads compressed .pt files and re-saves them uncompressed."""
    unzipped_path = preprocessed_path + "_unzipped"
    os.makedirs(unzipped_path, exist_ok=True)

    for file in tqdm(os.listdir(preprocessed_path), desc=" Unzipping .pt files"):
        if file.endswith(".pt"):
            file_path = os.path.join(preprocessed_path, file)
            save_path = os.path.join(unzipped_path, file)

            try:
                #  Load compressed file
                data = torch.load(file_path, map_location="cpu")

                #  Re-save uncompressed file
                torch.save(data, save_path, _use_new_zipfile_serialization=False)

                print(f" Unzipped and saved: {save_path}")

            except Exception as e:
                print(f" Failed to unzip {file}: {e}")

    print(f" All .pt files unzipped to: {unzipped_path}")
    return unzipped_path


In [ ]:
# Define your preprocessed EEG files
unzipped_path = "/data/ayesha/PhD/dataset/EEG/preprocessed_unzipped"
preprocessed_data = {
    f.replace("_preprocessed.pt", ""): os.path.join(unzipped_path, f)
    for f in os.listdir(unzipped_path) if f.endswith(".pt")
}

# Run Feature Extraction
save_features(preprocessed_data)

? All EEG files have already been processed! No new processing required.
